In [ ]:
from elasticsearch import Elasticsearch
import csv

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

with open('books_info.csv','r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        es.index(index='books1_info', body=row)

In [ ]:
!pip install cassandra-driver

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import pandas as pd

es = Elasticsearch(host='localhost', port=9200)

def get_data_from_elastic():
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "exists": {
                            "field": "children"
                        }
                    },
                    {
                        "regexp": {
                            "children": ".+"
                        }
                    }
                ]
            }
        }
    }

    rel = scan(
        client=es,
        query=query,
        scroll='1m',
        index='books_info',
        raise_on_error=True,
        preserve_order=False,
        clear_scroll=True
    )

    result = list(rel)

    temp = []

    for hit in result:
        temp.append(hit['_source'])

    df2 = pd.DataFrame(temp)

    return df2

df2 = get_data_from_elastic()

print(df2.head())
df2

                                               title   
0  Henderson's Australian families : a genealogic...  \
1  Abstract and analysis of the evidence taken by...   
2  The Burke and Wills exploring expedition : an ...   
3                 The Life and art of Walter Withers   
4  Private violence and public policy : the needs...   

                                      url                     contributors   
0  https://trove.nla.gov.au/work/11037485             Henderson, Alexander  \
1  https://trove.nla.gov.au/work/11179694    Lang, John Dunmore, 1799-1878   
2  https://trove.nla.gov.au/work/11299937  Burke, Robert O'Hara, 1821-1861   
3  https://trove.nla.gov.au/work/11313044              McCubbin, Alexander   
4  https://trove.nla.gov.au/work/11364357                       Pahl, J. M   

        date                              format   
0  1941-2004   Book|Book/Illustrated|Unpublished  \
1       1857  Book|Article/Other article|Article   
2       1861               Book/Illust

,title,url,contributors,date,format,fulltext_url,trove_id,language,rights,pages,form,volume,parent,children,text_downloaded,text_file
0,Henderson's Australian families : a genealogic...,https://trove.nla.gov.au/work/11037485,"Henderson, Alexander",1941-2004,Book|Book/Illustrated|Unpublished,http://nla.gov.au/nla.obj-77324653,nla.obj-77324653,English,In Copyright,0,Multi Volume Book,,,nla.obj-77324219,False,
1,Abstract and analysis of the evidence taken by...,https://trove.nla.gov.au/work/11179694,"Lang, John Dunmore, 1799-1878",1857,Book|Article/Other article|Article,http://nla.gov.au/nla.obj-412253504,nla.obj-412253504,,,0,Multi Volume Book,,,nla.obj-412255342|nla.obj-75124485,False,
2,The Burke and Wills exploring expedition : an ...,https://trove.nla.gov.au/work/11299937,"Burke, Robert O'Hara, 1821-1861",1861,Book/Illustrated|Book,http://nla.gov.au/nla.obj-396866331,nla.obj-396866331,,,0,Multi Volume Book,,,nla.obj-396883876|nla.obj-61875251,False,
3,The Life and art of Walter Withers,https://trove.nla.gov.au/work/11313044,"McCubbin, Alexander",1919-2003,Book|Book/Illustrated,http://nla.gov.au/nla.obj-375260165,nla.obj-375260165,English,No known copyright restrictions,0,Multi Volume Book,,,nla.obj-375325055|nla.obj-42322220,False,
4,Private violence and public policy : the needs...,https://trove.nla.gov.au/work/11364357,"Pahl, J. M",1873-2016,Book,http://nla.gov.au/nla.obj-753915387,nla.obj-753915387,English,No known copyright restrictions,0,Multi Volume Book,,,nla.obj-753915500,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Five years on : implementation of the Commonwe...,https://trove.nla.gov.au/work/8482284,Australia. Aboriginal and Torres Strait Island...,1997,Book|Book/Illustrated|Article,https://nla.gov.au/nla.obj-1297647186,nla.obj-1297647186,,,0,Multi Volume Book,,,nla.obj-1194416238|nla.obj-1194416440|nla.obj-...,False,
236,Izesuq oko moloko imolaqu,https://trove.nla.gov.au/work/9389532,,1971,Book,https://nla.gov.au/nla.obj-1385695208,nla.obj-1385695208,,,0,Multi Volume Book,,,nla.obj-1385727717|nla.obj-1385745492|nla.obj-...,False,
237,The cyclopedia of South Australia in two volum...,https://trove.nla.gov.au/work/9554744,"Burgess, H. T. (Henry Thomas), 1839-1923",1907-1978,Book/Illustrated|Book,http://nla.gov.au/nla.obj-63407750,nla.obj-63407750,,,0,Multi Volume Book,,,nla.obj-63407775|nla.obj-74680162,False,
238,Review of Australia's international civil avia...,https://trove.nla.gov.au/work/9574934,Australia. Review Committee on Australia's Int...,1978,Book|Government publication|Article,https://nla.gov.au/nla.obj-1335763041,nla.obj-1335763041,English,Out of Copyright,0,Multi Volume Book,,,nla.obj-1351153218|nla.obj-1351153613,False,


In [2]:
df2['year'] = df2['date'].str.extract(r'^(\d{4})')
df2

,title,url,contributors,date,format,fulltext_url,trove_id,language,rights,pages,form,volume,parent,children,text_downloaded,text_file,year
0,Henderson's Australian families : a genealogic...,https://trove.nla.gov.au/work/11037485,"Henderson, Alexander",1941-2004,Book|Book/Illustrated|Unpublished,http://nla.gov.au/nla.obj-77324653,nla.obj-77324653,English,In Copyright,0,Multi Volume Book,,,nla.obj-77324219,False,,1941
1,Abstract and analysis of the evidence taken by...,https://trove.nla.gov.au/work/11179694,"Lang, John Dunmore, 1799-1878",1857,Book|Article/Other article|Article,http://nla.gov.au/nla.obj-412253504,nla.obj-412253504,,,0,Multi Volume Book,,,nla.obj-412255342|nla.obj-75124485,False,,1857
2,The Burke and Wills exploring expedition : an ...,https://trove.nla.gov.au/work/11299937,"Burke, Robert O'Hara, 1821-1861",1861,Book/Illustrated|Book,http://nla.gov.au/nla.obj-396866331,nla.obj-396866331,,,0,Multi Volume Book,,,nla.obj-396883876|nla.obj-61875251,False,,1861
3,The Life and art of Walter Withers,https://trove.nla.gov.au/work/11313044,"McCubbin, Alexander",1919-2003,Book|Book/Illustrated,http://nla.gov.au/nla.obj-375260165,nla.obj-375260165,English,No known copyright restrictions,0,Multi Volume Book,,,nla.obj-375325055|nla.obj-42322220,False,,1919
4,Private violence and public policy : the needs...,https://trove.nla.gov.au/work/11364357,"Pahl, J. M",1873-2016,Book,http://nla.gov.au/nla.obj-753915387,nla.obj-753915387,English,No known copyright restrictions,0,Multi Volume Book,,,nla.obj-753915500,False,,1873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Five years on : implementation of the Commonwe...,https://trove.nla.gov.au/work/8482284,Australia. Aboriginal and Torres Strait Island...,1997,Book|Book/Illustrated|Article,https://nla.gov.au/nla.obj-1297647186,nla.obj-1297647186,,,0,Multi Volume Book,,,nla.obj-1194416238|nla.obj-1194416440|nla.obj-...,False,,1997
236,Izesuq oko moloko imolaqu,https://trove.nla.gov.au/work/9389532,,1971,Book,https://nla.gov.au/nla.obj-1385695208,nla.obj-1385695208,,,0,Multi Volume Book,,,nla.obj-1385727717|nla.obj-1385745492|nla.obj-...,False,,1971
237,The cyclopedia of South Australia in two volum...,https://trove.nla.gov.au/work/9554744,"Burgess, H. T. (Henry Thomas), 1839-1923",1907-1978,Book/Illustrated|Book,http://nla.gov.au/nla.obj-63407750,nla.obj-63407750,,,0,Multi Volume Book,,,nla.obj-63407775|nla.obj-74680162,False,,1907
238,Review of Australia's international civil avia...,https://trove.nla.gov.au/work/9574934,Australia. Review Committee on Australia's Int...,1978,Book|Government publication|Article,https://nla.gov.au/nla.obj-1335763041,nla.obj-1335763041,English,Out of Copyright,0,Multi Volume Book,,,nla.obj-1351153218|nla.obj-1351153613,False,,1978


In [3]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import pandas as pd

es = Elasticsearch(host='localhost', port=9200)

def get_data_from_elastic():
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "exists": {
                            "field": "parent"
                        }
                    },
                    {
                        "regexp": {
                            "parent": ".+"
                        }
                    }
                ]
            }
        }
    }

    rel = scan(
        client=es,
        query=query,
        scroll='1m',
        index='books_info',
        raise_on_error=True,
        preserve_order=False,
        clear_scroll=True
    )

    result = list(rel)

    temp = []

    for hit in result:
        temp.append(hit['_source'])

    df3 = pd.DataFrame(temp)

    return df3

df3 = get_data_from_elastic()

print(df3.head())
df3

                                               title   
0  Henderson's Australian families : a genealogic...  \
1  Abstract and analysis of the evidence taken by...   
2  Abstract and analysis of the evidence taken by...   
3  The Burke and Wills exploring expedition : an ...   
4  The Burke and Wills exploring expedition : an ...   

                                      url                     contributors   
0  https://trove.nla.gov.au/work/11037485             Henderson, Alexander  \
1  https://trove.nla.gov.au/work/11179694    Lang, John Dunmore, 1799-1878   
2  https://trove.nla.gov.au/work/11179694    Lang, John Dunmore, 1799-1878   
3  https://trove.nla.gov.au/work/11299937  Burke, Robert O'Hara, 1821-1861   
4  https://trove.nla.gov.au/work/11299937  Burke, Robert O'Hara, 1821-1861   

        date                              format   
0  1941-2004   Book|Book/Illustrated|Unpublished  \
1       1857  Book|Article/Other article|Article   
2       1857  Book|Article/Other artic

,title,url,contributors,date,format,fulltext_url,trove_id,language,rights,pages,form,volume,parent,children,text_downloaded,text_file
0,Henderson's Australian families : a genealogic...,https://trove.nla.gov.au/work/11037485,"Henderson, Alexander",1941-2004,Book|Book/Illustrated|Unpublished,http://nla.gov.au/nla.obj-77324219,nla.obj-77324219,English,In Copyright,453,Multi Volume Book,1,nla.obj-77324653,,True,henderson-s-australian-families-a-genealogical...
1,Abstract and analysis of the evidence taken by...,https://trove.nla.gov.au/work/11179694,"Lang, John Dunmore, 1799-1878",1857,Book|Article/Other article|Article,http://nla.gov.au/nla.obj-412255342,nla.obj-412255342,,,104,Multi Volume Book,1,nla.obj-412253504,,True,abstract-and-analysis-of-the-evidence-taken-by...
2,Abstract and analysis of the evidence taken by...,https://trove.nla.gov.au/work/11179694,"Lang, John Dunmore, 1799-1878",1857,Book|Article/Other article|Article,http://nla.gov.au/nla.obj-75124485,nla.obj-75124485,,,104,Multi Volume Book,2,nla.obj-412253504,,True,abstract-and-analysis-of-the-evidence-taken-by...
3,The Burke and Wills exploring expedition : an ...,https://trove.nla.gov.au/work/11299937,"Burke, Robert O'Hara, 1821-1861",1861,Book/Illustrated|Book,http://nla.gov.au/nla.obj-396883876,nla.obj-396883876,,,46,Multi Volume Book,1,nla.obj-396866331,,True,the-burke-and-wills-exploring-expedition-an-ac...
4,The Burke and Wills exploring expedition : an ...,https://trove.nla.gov.au/work/11299937,"Burke, Robert O'Hara, 1821-1861",1861,Book/Illustrated|Book,http://nla.gov.au/nla.obj-61875251,nla.obj-61875251,,,46,Multi Volume Book,2,nla.obj-396866331,,True,the-burke-and-wills-exploring-expedition-an-ac...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,The cyclopedia of South Australia in two volum...,https://trove.nla.gov.au/work/9554744,"Burgess, H. T. (Henry Thomas), 1839-1923",1907-1978,Book/Illustrated|Book,http://nla.gov.au/nla.obj-74680162,nla.obj-74680162,,,1124,Journal,2,nla.obj-63407750,,True,the-cyclopedia-of-south-australia-in-two-volum...
1419,Review of Australia's international civil avia...,https://trove.nla.gov.au/work/9574934,Australia. Review Committee on Australia's Int...,1978,Book|Government publication|Article,http://nla.gov.au/nla.obj-1351153218,nla.obj-1351153218,English,Out of Copyright,288,Multi Volume Book,1,nla.obj-1335763041,,True,review-of-australia-s-international-civil-avia...
1420,Review of Australia's international civil avia...,https://trove.nla.gov.au/work/9574934,Australia. Review Committee on Australia's Int...,1978,Book|Government publication|Article,http://nla.gov.au/nla.obj-1351153613,nla.obj-1351153613,English,Out of Copyright,448,Multi Volume Book,2,nla.obj-1335763041,,True,review-of-australia-s-international-civil-avia...
1421,The progress of Victoria : a statistical essay...,https://trove.nla.gov.au/work/9800197,"Archer, William Henry, 1825-1909",1867-1873,Book|Government publication|Book/Illustrated,http://nla.gov.au/nla.obj-398086951,nla.obj-398086951,English,Out of Copyright|http://rightsstatements.org/v...,108,Multi Volume Book,1,nla.obj-396862927,,True,the-progress-of-victoria-a-statistical-essay-b...


In [4]:
df3['year'] = df3['date'].str.extract(r'^(\d{4})')
df3

,title,url,contributors,date,format,fulltext_url,trove_id,language,rights,pages,form,volume,parent,children,text_downloaded,text_file,year
0,Henderson's Australian families : a genealogic...,https://trove.nla.gov.au/work/11037485,"Henderson, Alexander",1941-2004,Book|Book/Illustrated|Unpublished,http://nla.gov.au/nla.obj-77324219,nla.obj-77324219,English,In Copyright,453,Multi Volume Book,1,nla.obj-77324653,,True,henderson-s-australian-families-a-genealogical...,1941
1,Abstract and analysis of the evidence taken by...,https://trove.nla.gov.au/work/11179694,"Lang, John Dunmore, 1799-1878",1857,Book|Article/Other article|Article,http://nla.gov.au/nla.obj-412255342,nla.obj-412255342,,,104,Multi Volume Book,1,nla.obj-412253504,,True,abstract-and-analysis-of-the-evidence-taken-by...,1857
2,Abstract and analysis of the evidence taken by...,https://trove.nla.gov.au/work/11179694,"Lang, John Dunmore, 1799-1878",1857,Book|Article/Other article|Article,http://nla.gov.au/nla.obj-75124485,nla.obj-75124485,,,104,Multi Volume Book,2,nla.obj-412253504,,True,abstract-and-analysis-of-the-evidence-taken-by...,1857
3,The Burke and Wills exploring expedition : an ...,https://trove.nla.gov.au/work/11299937,"Burke, Robert O'Hara, 1821-1861",1861,Book/Illustrated|Book,http://nla.gov.au/nla.obj-396883876,nla.obj-396883876,,,46,Multi Volume Book,1,nla.obj-396866331,,True,the-burke-and-wills-exploring-expedition-an-ac...,1861
4,The Burke and Wills exploring expedition : an ...,https://trove.nla.gov.au/work/11299937,"Burke, Robert O'Hara, 1821-1861",1861,Book/Illustrated|Book,http://nla.gov.au/nla.obj-61875251,nla.obj-61875251,,,46,Multi Volume Book,2,nla.obj-396866331,,True,the-burke-and-wills-exploring-expedition-an-ac...,1861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,The cyclopedia of South Australia in two volum...,https://trove.nla.gov.au/work/9554744,"Burgess, H. T. (Henry Thomas), 1839-1923",1907-1978,Book/Illustrated|Book,http://nla.gov.au/nla.obj-74680162,nla.obj-74680162,,,1124,Journal,2,nla.obj-63407750,,True,the-cyclopedia-of-south-australia-in-two-volum...,1907
1419,Review of Australia's international civil avia...,https://trove.nla.gov.au/work/9574934,Australia. Review Committee on Australia's Int...,1978,Book|Government publication|Article,http://nla.gov.au/nla.obj-1351153218,nla.obj-1351153218,English,Out of Copyright,288,Multi Volume Book,1,nla.obj-1335763041,,True,review-of-australia-s-international-civil-avia...,1978
1420,Review of Australia's international civil avia...,https://trove.nla.gov.au/work/9574934,Australia. Review Committee on Australia's Int...,1978,Book|Government publication|Article,http://nla.gov.au/nla.obj-1351153613,nla.obj-1351153613,English,Out of Copyright,448,Multi Volume Book,2,nla.obj-1335763041,,True,review-of-australia-s-international-civil-avia...,1978
1421,The progress of Victoria : a statistical essay...,https://trove.nla.gov.au/work/9800197,"Archer, William Henry, 1825-1909",1867-1873,Book|Government publication|Book/Illustrated,http://nla.gov.au/nla.obj-398086951,nla.obj-398086951,English,Out of Copyright|http://rightsstatements.org/v...,108,Multi Volume Book,1,nla.obj-396862927,,True,the-progress-of-victoria-a-statistical-essay-b...,1867


In [12]:
df2['year'] = df2['year'].fillna(0)

In [13]:
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

cluster = Cluster(['localhost'])
session = cluster.connect()

session.execute("CREATE KEYSPACE IF NOT EXISTS my_keyspace WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.execute("USE my_keyspace")

query = "CREATE TABLE IF NOT EXISTS books_children2 (trove_id text PRIMARY KEY, title text, contributors text, url text,date text, format text, year int)"
session.execute(query)

prepared = session.prepare("INSERT INTO books_children2 (trove_id, title, contributors, url,date, format, year) VALUES (?, ?, ?, ?, ?, ?, ?)")

batch_size = 100
num_batches = len(df2) // batch_size + 1

for i in range(num_batches):
    batch = BatchStatement()
    start_index = i * batch_size
    end_index = (i + 1) * batch_size
    for index, row in df2.iloc[start_index:end_index].iterrows():
#         batch.add(prepared, (row['trove_id'], row['title'], row['contributors'], row['url'],row['date'], row['format'],row['year']))
        batch.add(prepared, (row['trove_id'], row['title'], row['contributors'], row['url'], row['date'], row['format'], int(row['year'])))
    session.execute(batch)


In [14]:
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'])
session = cluster.connect()

session.execute("USE my_keyspace")

query = "SELECT * FROM books_children2"
result = session.execute(query)

for row in result:
    print(row)


Row(trove_id='nla.obj-33236945', contributors='Society for Nautical Research (London, England)', date='1935-1938', format='Book|Book/Illustrated', title='Lists of men-of-war, 1650-1700', url='https://trove.nla.gov.au/work/13097671', year=1935)
Row(trove_id='nla.obj-39267340', contributors='Campbell, Bessie, 1870-1964', date='1889', format='Book', title='[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career]', url='https://trove.nla.gov.au/work/237278882', year=1889)
Row(trove_id='nla.obj-475664029', contributors='', date='1860', format='Government publication|Book', title='Report of the proceedings of the Queensland Government schooner "Spitfire" in search of the mouth of the River Burdekin, on the north-eastern coast of Australia : and of the exploration of a portion of that coast extending from Gloucester Island to Halifax Bay', url='https://trove.nla.gov.au/work/18245206', year=1860)
Row(trove_id='nla.obj-410740363', contributors='', date=

In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

cluster = Cluster(['localhost'])
session = cluster.connect()

session.execute("CREATE KEYSPACE IF NOT EXISTS my_keyspace WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.execute("USE my_keyspace")

query = "CREATE TABLE IF NOT EXISTS books_parent1 (trove_id text PRIMARY KEY, title text, contributors text, url text,date text, format text)"
session.execute(query)

prepared = session.prepare("INSERT INTO books_parent1 (trove_id, title, contributors, url,date, format) VALUES (?, ?, ?, ?, ?,?)")

batch_size = 100
num_batches = len(df3) // batch_size + 1

for i in range(num_batches):
    batch = BatchStatement()
    start_index = i * batch_size
    end_index = (i + 1) * batch_size
    for index, row in df3.iloc[start_index:end_index].iterrows():
        batch.add(prepared, (row['trove_id'], row['title'], row['contributors'], row['url'],row['date'], row['format']))
    session.execute(batch)


In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'])
session = cluster.connect()

session.execute("USE my_keyspace")

query = "SELECT * FROM books_parent1"
result = session.execute(query)

for row in result:
    print(row)


In [ ]:
from cassandra.query import SimpleStatement

query = "SELECT * FROM books_parent1"
statement = SimpleStatement(query)

rows = session.execute(statement)
# for row in rows:
#     date = row.date[:3]
#     if int(date) < 2000:
#         print(row)
for row in rows:
    date = row.date[:4]
    if date and int(date) < 2000:
        print(row)

In [18]:
# from cassandra.query import SimpleStatement

# query = "SELECT * FROM books_children2 "
# statement = SimpleStatement(query)

# rows = session.execute(statement)
# # for row in rows:
# #     date = row.date[:3]
# #     if int(date) < 2000:
# #         print(row)
# for row in rows:
#     date = row.date[:4]
#     if date and int(date) < 2000:
#         print(row)


# query_one = f"SELECT * FROM books_children2 WHERE year < '2000' ALLOW FILTERING;"
query_one = f"SELECT * FROM books_children2 WHERE year < {int('2000')} ALLOW FILTERING;"

result__one = session.execute(query_one)
for res in result__one:
    print(res)

Row(trove_id='nla.obj-33236945', contributors='Society for Nautical Research (London, England)', date='1935-1938', format='Book|Book/Illustrated', title='Lists of men-of-war, 1650-1700', url='https://trove.nla.gov.au/work/13097671', year=1935)
Row(trove_id='nla.obj-39267340', contributors='Campbell, Bessie, 1870-1964', date='1889', format='Book', title='[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career]', url='https://trove.nla.gov.au/work/237278882', year=1889)
Row(trove_id='nla.obj-475664029', contributors='', date='1860', format='Government publication|Book', title='Report of the proceedings of the Queensland Government schooner "Spitfire" in search of the mouth of the River Burdekin, on the north-eastern coast of Australia : and of the exploration of a portion of that coast extending from Gloucester Island to Halifax Bay', url='https://trove.nla.gov.au/work/18245206', year=1860)
Row(trove_id='nla.obj-410740363', contributors='', date=

In [10]:
import pandas as pd
null_counts = df2.isnull().sum()
print(null_counts)

title              0
url                0
contributors       0
date               0
format             0
fulltext_url       0
trove_id           0
language           0
rights             0
pages              0
form               0
volume             0
parent             0
children           0
text_downloaded    0
text_file          0
year               1
dtype: int64


In [11]:
import pandas as pd
null_counts = df3.isnull().sum()
print(null_counts)

title              0
url                0
contributors       0
date               0
format             0
fulltext_url       0
trove_id           0
language           0
rights             0
pages              0
form               0
volume             0
parent             0
children           0
text_downloaded    0
text_file          0
year               1
dtype: int64


In [ ]:
# from cassandra.query import SimpleStatement

# # query = "SELECT * FROM books_children1 "
# # statement = SimpleStatement(query)

# # rows = session.execute(statement)
# # # for row in rows:
# # #     date = row.date[:3]
# # #     if int(date) < 2000:
# # #         print(row)
# # for row in rows:
# #     date = row.date[:4]
# #     if date and int(date) < 2000:
# #         print(row)


# query_one = f"SELECT * FROM {books_children1 WHERE date < '2000' ALLOW FILTERING;"
# result__one = session.execute(query_one)
# for res in result__one:
#     print(res)